In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/15/23 We want to add:
#     Take into account the WIDTH/STIM LOCATION of every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
####################################################################################

In [15]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
cell_type_prefix = 'pv'
csv_prefix = cell_type_prefix + '_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg',
              'Age',
              'Infusion',
              'Sex',
              'Stim (uA)',
              'Delta latency 1 (barrel 1 - barrel 2)',
              'Delta latency 2 (barrel 3 - barrel 2)']

enable_photoZ_interaction = True
initialize_photoZ = False
snr_cutoff_signal = 5.0

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/" + cell_type_prefix +"_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,ROI jpg,Age,Infusion,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2),abs(delta 1),abs(delta 2)
0,2/28/2023,02_01_01,L5,50.3009,50.8406,49.3457,2.940390,10.630900,6.484390,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-1.4949,NaN,1.4949
1,2/28/2023,02_02_02,L5,49.9882,51.3212,51.1805,2.045210,5.142090,2.921330,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,NaN,NaN,NaN
2,2/28/2023,03_01_01,L2/3,52.5719,51.2136,52.9013,9.405090,10.237200,5.780930,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.3583,1.6877,1.3583,1.6877
3,2/28/2023,03_02_02,L2/3,51.8399,50.0310,50.7536,14.738200,35.400200,21.883200,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.8089,0.7226,1.8089,0.7226
4,2/28/2023,06_01_01,L5,51.1367,51.0128,50.4326,5.726470,13.058300,13.063500,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,0.1239,-0.5802,0.1239,0.5802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,6/28/2023,02_04_03,L4,58.3278,94.0000,94.0000,1.284740,0.000000,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
86,6/28/2023,03_01_10,L4,94.0000,51.1749,51.3676,0.000000,0.263371,1.110580,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
87,6/28/2023,03_02_03,L4,58.9868,58.3472,94.0000,0.077615,0.514122,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN
88,6/28/2023,03_03_01,L4,50.7398,94.0000,94.0000,0.862532,0.000000,0.000000,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,83,GABAzine,F,200,NaN,NaN,NaN,NaN


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'slice' in f[-17:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)


        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from cell_type_prefix_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        
        # open these various files and store them to the pandas dataframe
        # The first three ROIs of each should correspond to the 3 desired barrel ROIs
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        print(new_row)
        df = df.append(new_row, ignore_index=True)




scraping data from  01_01_03.zda using slice1_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '01_01_03', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 58.4733, 'L4 barrel 2 Latency': 51.3893, 'L4 barrel 3 Latency': 51.6462, 'L4 barrel 1 SNR': 0.408783, 'L4 barrel 2 SNR': 1.71941, 'L4 barrel 3 SNR': 1.49933, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_01_01_03.jpg'}


scraping data from  01_02_03.zda using slice1_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '01_02_03', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 51.0282, 'L4 barrel 2 Latency': 51.4216, 'L4 barrel 3 Latency': 56.7903, 'L4 barrel 1 SNR': 2.04579, 'L4 barrel 2 SNR': 1.62347, 'L4 barrel 3 SNR': 0.405088, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_01_02_03.jpg'}


scraping data from  01_03_02.zda using sli



scraping data from  04_02_03.zda using slice4_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '04_02_03', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 51.0595, 'L4 barrel 2 Latency': 50.3504, 'L4 barrel 3 Latency': 51.0249, 'L4 barrel 1 SNR': 3.55902, 'L4 barrel 2 SNR': 9.11482, 'L4 barrel 3 SNR': 6.24122, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_04_02_03.jpg'}


scraping data from  04_03_03.zda using slice4_rois.dat on date: 6/26/2023
{'Date': '6/26/2023', 'Slice/Loc/Rec': '04_03_03', 'Age': 81, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 50.4028, 'L4 barrel 2 Latency': 51.2266, 'L4 barrel 3 Latency': 53.8788, 'L4 barrel 1 SNR': 6.87042, 'L4 barrel 2 SNR': 5.08688, 'L4 barrel 3 SNR': 2.42056, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-26-23\\selected_zda/ROIs_04_03_03.jpg'}


scraping data from  04_04_03.zda using slice



scraping data from  03_02_03.zda using slice3_rois.dat on date: 6/28/2023
{'Date': '6/28/2023', 'Slice/Loc/Rec': '03_02_03', 'Age': 83, 'Sex': 'F', 'Infusion': 'GABAzine', 'Stim (uA)': '200', 'L4 barrel 1 Latency': 58.9868, 'L4 barrel 2 Latency': 58.3472, 'L4 barrel 3 Latency': 94.0, 'L4 barrel 1 SNR': 0.0776151, 'L4 barrel 2 SNR': 0.514122, 'L4 barrel 3 SNR': 0.0, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-28-23\\selected_zda/ROIs_03_02_03.jpg'}


scraping data from  03_03_01.zda using slice3_rois.dat on date: 6/28/2023
{'Date': '6/28/2023', 'Slice/Loc/Rec': '03_03_01', 'Age': 83, 'Sex': 'F', 'Infusion': 'GABAzine', 'Stim (uA)': '200', 'L4 barrel 1 Latency': 50.7398, 'L4 barrel 2 Latency': 94.0, 'L4 barrel 3 Latency': 94.0, 'L4 barrel 1 SNR': 0.862532, 'L4 barrel 2 SNR': 0.0, 'L4 barrel 3 SNR': 0.0, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-28-23\\selected_zda/ROIs_03_03_01.jpg'}


scraping data from  03_04_03.zda using slice3_r

In [16]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

Barrel pairs in L2/3
	 11 no crossing out of 68 in layer L2/3
	 14 crossing out of 68 in layer L2/3
	 43 no signal out of 68 in layer L2/3
Barrel pairs in L4
	 29 no crossing out of 88 in layer L4
	 14 crossing out of 88 in layer L4
	 45 no signal out of 88 in layer L4
Barrel pairs in L5
	 11 no crossing out of 18 in layer L5
	 5 crossing out of 18 in layer L5
	 2 no signal out of 18 in layer L5


In [10]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

Latency between barrel pairs in L2/3
	 0.930 +/- 0.579 (n = 22)
	 [1.3583 1.4885 1.8089 0.5078 0.5782 0.8922 0.7235 1.1776 0.4841 1.8474
 0.2455 0.0147 1.6877 0.7967 0.7226 1.9021 0.7541 0.2124 1.7317 0.433
 0.7534 0.3538]
Latency between barrel pairs in L4
	 0.846 +/- 0.995 (n = 27)
	 [0.1687 1.0151 1.3033 2.096  0.826  0.012  3.7841 0.5839 0.456  0.8266
 0.477  0.2671 0.0656 0.0797 0.0584 0.0353 0.4992 0.8238 1.2135 0.2154
 3.7901 0.03   0.8443 0.4909 0.6745 2.048  0.1808]
Latency between barrel pairs in L5
	 0.726 +/- 0.446 (n = 13)
	 [0.8933 0.1239 1.4173 0.5098 0.0111 1.4949 0.9943 0.784  0.5802 1.0574
 0.1154 0.724  0.7415]


In [14]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [ ]:
df